In [ ]:
import os
import torch
from torch import nn  # 뉴럴네트워크 함수
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [ ]:
# torch는 다 벡터 값이다.

In [ ]:
## 딥러닝 모델

In [ ]:
# 뉴럴네트워크
class NeuralNet(nn.Module): # 뉴럴네트워크 만들때 클래스에 nn.Module을 만드는게 국룰이다. 
    def __init__(self, input_size, hidden_size, num_classes):  # 초기값 설정하는 것. self뒤에 것들은 class에 input으로 넣을 것을 적어주는 것이다. 그리고 num_classes는 최종 출력값이다. 
        super(NeuralNet, self).__init__()  # 이거는 왜 하는 걸까?
        self.mlp1 = nn.Linear(input_size, hidden_size) # 첫번째 layer FC이다.
        self.relu = nn.ReLU()  # 활성함수 지정
        self.mlp2 = nn.Linear(hidden_size, num_classes) # 두번째 layer FC이다. 
        self.softmax = nn.Softmax(dim=1) # 분류작업 하기위해 소프트맥스로 확률값을 취하기 위해 쓴다. 
    def forward(self, x):  # 출력값 설정
        out = self.mlp1(x)
        out = self.relu(out)
        out = self.mlp2(out)
        out = self.softmax(out)
        return out

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [ ]:
input_size = 28*28*1 # MNIST 이미지 크기
hidden_size = 100 # hyper parameter
num_classes = 10 # 총 MNIST의 class 갯수

model = NeuralNet(input_size, hidden_size, num_classes).to(device)   # .to(device)는 cpu 일때 안써도 된다.
print(model)

NeuralNet(
  (mlp1): Linear(in_features=784, out_features=100, bias=True)
  (relu): ReLU()
  (mlp2): Linear(in_features=100, out_features=10, bias=True)
  (softmax): Softmax(dim=1)
)


In [ ]:
data = torch.rand(1, 28*28).to(device)
print('data:',data.shape)
print('-'*40)

pred = model(data)
print('pred:',pred)
print('-'*40)

y_hat = pred.argmax(1) # 값이 제일 높은거 하나 뽑기
print('y hat:',y_hat)

data: torch.Size([1, 784])
----------------------------------------
pred: tensor([[0.0907, 0.0983, 0.1043, 0.0927, 0.0947, 0.1213, 0.0872, 0.1213, 0.0953,
         0.0943]], grad_fn=<SoftmaxBackward>)
----------------------------------------
y hat: tensor([7])


### nn.Linear

In [ ]:
x = torch.randn(3,28,28)  # 3은 데이터의 갯수이다. 28x28 이미지크기

In [ ]:
fc_layer = nn.Linear(in_features=28*28, out_features=100)
x_after_fc = fc_layer(x.reshape(-1,28*28))
print('x_after_fc:',x_after_fc.shape)

x_after_fc: torch.Size([3, 100])


In [ ]:
# nn.Flatten : x.reshape(-1,28*28)과 같은 역할
# 28x28 사이즈의 이미지를 784 픽셀 값을 갖는 배열로 변환

flatten = nn.Flatten()
flat_x = flatten(x)
print(flat_x.shape)

fc_layer = nn.Linear(in_features=28*28, out_features=100)
x_after_fc = fc_layer(flat_x)
print('x_after_fc:',x_after_fc.shape)

torch.Size([3, 784])
x_after_fc: torch.Size([3, 100])


### nn.ReLU

In [ ]:
x = torch.randn(3,28,28)  # 3은 데이터 갯수이고, 28x28 행렬로 이루어져있다.

In [ ]:
before_relu = x[0][0]
print('before_relu :', before_relu, before_relu.shape)
print('-'*100)

relu = nn.ReLU()
after_relu = relu(before_relu)
print('after_relu :', after_relu, after_relu.shape)

before_relu : tensor([ 1.3258, -1.0004, -1.2106, -0.1239,  1.1380, -0.4152, -0.5813, -0.3026,
        -0.4818, -1.3361,  1.8368,  1.0988,  0.6355, -0.9385, -0.1672, -0.4743,
        -1.5085,  0.6050,  1.1130,  0.4190, -0.5781, -0.1485,  0.8649, -1.6233,
         0.8800, -0.7283, -1.7633,  0.1791]) torch.Size([28])
----------------------------------------------------------------------------------------------------
after_relu : tensor([1.3258, 0.0000, 0.0000, 0.0000, 1.1380, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 1.8368, 1.0988, 0.6355, 0.0000, 0.0000, 0.0000, 0.0000, 0.6050,
        1.1130, 0.4190, 0.0000, 0.0000, 0.8649, 0.0000, 0.8800, 0.0000, 0.0000,
        0.1791]) torch.Size([28])


### Softmax

In [ ]:
x = torch.randn(1,10)  # 
x

tensor([[ 0.1237, -0.7806, -1.1575, -1.2096, -0.0131,  0.2782,  0.8502,  1.2090,
         -0.0721, -1.8007]])

In [ ]:
softmax = nn.Softmax(dim=1)

In [ ]:
pred = softmax(x)
print('pred:', pred)
print('sum of pred:',pred.sum())

pred: tensor([[0.1002, 0.0406, 0.0278, 0.0264, 0.0874, 0.1169, 0.2072, 0.2966, 0.0824,
         0.0146]])
sum of pred: tensor(1.)


### nn.Sequential

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.mlp1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.mlp2 = nn.Linear(hidden_size, num_classes)  
        self.softmax = nn.Softmax(dim=1)  # 1차원에 대해서 적용
    def forward(self, x):
        out = self.mlp1(x)
        out = self.relu(out)
        out = self.mlp2(out)
        out = self.softmax(out)
        return out

In [ ]:
input_size = 28*28*1 # MNIST 이미지 크기
hidden_size = 100 # hyper parameter
num_classes = 10 # 총 class 수

model = NeuralNet(input_size, hidden_size, num_classes).to(device)
print(model)

NeuralNet(
  (mlp1): Linear(in_features=784, out_features=100, bias=True)
  (relu): ReLU()
  (mlp2): Linear(in_features=100, out_features=10, bias=True)
  (softmax): Softmax(dim=1)
)


In [ ]:
# nn.Sequential : 여러 모듈들을 묶어서 사용 가능

class NeuralNet2(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        sequential = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, num_classes),
            nn.Softmax(dim=1),
        )
    def forward(self, x):
        out = self.sequential(x)
        return out

In [ ]:
input_size = 28*28*1 # MNIST 이미지 크기
hidden_size = 100 # hyper parameter
num_classes = 10 # 총 class 수

model = NeuralNet(input_size, hidden_size, num_classes).to(device)
print(model)

NeuralNet(
  (mlp1): Linear(in_features=784, out_features=100, bias=True)
  (relu): ReLU()
  (mlp2): Linear(in_features=100, out_features=10, bias=True)
  (softmax): Softmax(dim=1)
)


## 손실 함수

### Cross Entropy

Binary Cross Entropy Loss (nn.BCELoss())
- Binary Class (0/1)에 적용
- Loss 적용 전 sigmoid나 softmax를 취해줘야 함

In [ ]:
x = torch.randn(3)
y = torch.tensor([0.,1.,0.]) # binary 이므로 target(y) 값은 0 또는 1

In [ ]:
print('x:',x)
print('y:',y)

x: tensor([-0.7477,  0.4827,  0.6035])
y: tensor([0., 1., 0.])


In [ ]:
# Binary Cross Entropy Loss
bce_loss = nn.BCELoss()
sigmoid = nn.Sigmoid()

x_sigmoid = sigmoid(x)
print('binary cross entropy loss value:', bce_loss(x_sigmoid,y))

binary cross entropy loss value: tensor(0.6360)


Binary Cross Entropy With Logits Loss (nn.BCEWithLogitsLoss())
- Binary Class (0/1)에 적용
- Loss안에 Sigmoid가 내장되어있음

In [ ]:
# Binary Cross Entropy Loss
bce_with_logits_loss = nn.BCEWithLogitsLoss()

print('binary cross entropy loss value:', bce_with_logits_loss(x,y))

binary cross entropy loss value: tensor(0.6360)


Cross Entropy Loss (nn.CrossEntropyLoss())
- Multi Class에서 적용
- Loss안에 Softmax가 내장되어 있음

In [ ]:
x = torch.randn(1,10)
y = torch.tensor([1])

In [ ]:
print('x:',x)
print('y:',y)

x: tensor([[-1.3782,  0.8480, -0.1404,  1.0514, -1.0601,  0.1745,  0.1179, -1.1434,
         -0.2724, -1.2614]])
y: tensor([1])


In [ ]:
# Cross Entropy Loss

cross_entropy_loss = nn.CrossEntropyLoss()
# Softmax와 Log를 한 후 Cross Entropy Loss

print('cross_entropy loss value:', cross_entropy_loss(x,y))

cross_entropy loss value: tensor(1.4884)


### MSE

In [ ]:
x = torch.randn(1,10)  # 리그레션에 해당하는 loss이기에 꼭 정수일 필요없다.
y = torch.randn(1,10)

In [ ]:
mse_loss = nn.MSELoss()
print('cross_entropy loss value:', mse_loss(x,y))

cross_entropy loss value: tensor(1.4131)
